In [1]:
# Subjectivitiy 
# Packages einlesen
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np

In [6]:
data_hedonic = pd.read_csv("/Users/paulahofmann/Documents/Coding/Online-Review/FeaturePreperation/Data_with_Features/Total_Features_utilitarian.csv")

In [8]:
import pandas as pd
from transformers import pipeline

# Define the text classification pipeline
classify = pipeline(
    task="text-classification",
    model="cffl/bert-base-styleclassification-subjective-neutral",
    top_k=None,  # Equivalent to return_all_scores=True
)

# Function to classify review text and return the scores for all labels
def classify_text(text):
    classification_result = classify(text)
    return classification_result[0]

# Function to process longer texts by splitting them into smaller chunks
def process_long_text(text, max_chunk_length=512):
    chunks = [text[i:i+max_chunk_length] for i in range(0, len(text), max_chunk_length)]
    scores = []
    for chunk in chunks:
        chunk_scores = classify_text(chunk)
        scores.extend(chunk_scores)
    return scores

# Apply text classification to each review text in the DataFrame and extract scores for all labels
data_hedonic['classification_scores'] = data_hedonic['text_cleaned'].apply(process_long_text)

# Extract subjective and neutral scores separately
subjective_scores = []
neutral_scores = []
for _, row in data_hedonic.iterrows():
    label_scores = row['classification_scores']
    subjective_score = next((entry['score'] for entry in label_scores if entry['label'] == 'SUBJECTIVE'), None)
    neutral_score = next((entry['score'] for entry in label_scores if entry['label'] == 'NEUTRAL'), None)
    subjective_scores.append(subjective_score)
    neutral_scores.append(neutral_score)

# Add the extracted scores as new columns to the DataFrame
data_hedonic['subjective_score'] = subjective_scores
data_hedonic['neutral_score'] = neutral_scores

# Drop the 'classification_scores' column as it's no longer needed
data_hedonic.drop(columns=['classification_scores'], inplace=True)

print(data_hedonic)


       rating                                            title_x  \
0         5.0                                      Affordability   
1         5.0                                          Great buy   
2         5.0              Good value - comparable to Angel Soft   
3         2.0                                       Not sanitary   
4         5.0                               Strong and absorbent   
...       ...                                                ...   
34052     5.0                                      Works wonders   
34053     5.0                                          Resultado   
34054     5.0                                         Amazing!!!   
34055     5.0    This brush has lifted a weight off my shoulders   
34056     5.0  Amazing! Effortlessly detangles without causin...   

                                                    text images        asin  \
0      I use more experience rolls but this is great ...     []  B095CN96JS   
1      I expected just to

In [9]:
data_hedonic.to_csv("/Users/paulahofmann/Documents/Coding/Online-Review/FeaturePreperation/Data_with_Features/Total_Features_Utilitarian_Subj.csv")